In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols

In [2]:
np.random.seed(42)

In [3]:
url = 'https://raw.githubusercontent.com/neurodatascience/course-2020-assessments/master/practical/brainsize.csv'
df = pd.read_csv(url, sep=';', na_values='.', index_col=0)

print(df.head(5))
print(len(df.index))

   Gender  FSIQ  VIQ  PIQ  Weight  Height  MRI_Count
1  Female   133  132  124   118.0    64.5     816932
2    Male   140  150  124     NaN    72.5    1001121
3    Male   139  123  150   143.0    73.3    1038437
4    Male   133  129  128   172.0    68.8     965353
5  Female   137  132  134   147.0    65.0     951545
40


In [4]:
partY = np.random.normal(loc=0.0, scale=1.0, size=len(df.index))
df.insert(len(df.columns), 'partY', partY)
print(df.head(5))

   Gender  FSIQ  VIQ  PIQ  Weight  Height  MRI_Count     partY
1  Female   133  132  124   118.0    64.5     816932  0.496714
2    Male   140  150  124     NaN    72.5    1001121 -0.138264
3    Male   139  123  150   143.0    73.3    1038437  0.647689
4    Male   133  129  128   172.0    68.8     965353  1.523030
5  Female   137  132  134   147.0    65.0     951545 -0.234153


In [5]:
not_partY = list(df.columns[0:len(df.columns)-1])
print(not_partY)

['Gender', 'FSIQ', 'VIQ', 'PIQ', 'Weight', 'Height', 'MRI_Count']


In [6]:
model = ols('partY ~ PIQ + Weight', data=df).fit()
model.summary()

In [7]:
new_df = df.drop(1)

In [8]:
model = ols('partY ~ PIQ + Weight', data=new_df).fit()
model.summary()